## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000254C4362600>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000254C6A7B1A0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Abhii Yadav and I am a Generative AI LLM Engineer")])

AIMessage(content="Hi Abhii Yadav, it's nice to meet you!\n\nThat's a fascinating field to work in.  As a large language model myself, I'm always interested to learn more about the people who build and develop AI like me.\n\nWhat kind of projects are you working on?  Are there any particular challenges you're facing in your work?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 27, 'total_tokens': 105, 'completion_time': 0.141818182, 'prompt_time': 0.002086456, 'queue_time': 0.264060366, 'total_time': 0.143904638}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--d9456495-6363-42c6-978b-8620083fc930-0', usage_metadata={'input_tokens': 27, 'output_tokens': 78, 'total_tokens': 105})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Abhii Yadav and I am a Generative AI LLM Engineer"),
        AIMessage(content="Hello Abhii! It's nice to meet you. \n\nAs a Generative AI LLM Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Abhii Yadav, a Generative AI LLM Engineer!  😊  \n\nIs there anything else you'd like me to remember about you?  Perhaps your favorite project or a fun fact?  I'm here to learn more.  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 106, 'total_tokens': 161, 'completion_time': 0.1, 'prompt_time': 0.018344049, 'queue_time': 0.512430081, 'total_time': 0.118344049}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--920b6286-1b98-4b5f-8a75-d7d128ce679e-0', usage_metadata={'input_tokens': 106, 'output_tokens': 55, 'total_tokens': 161})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
%pip install langchain_community


  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl (156 kB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\ADMIN\\anaconda3\\Lib\\site-packages\\yaml\\_yaml.cp312-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [ ]:
# This code sets up a chatbot using LangChain and Groq's LLM API.
# - It loads your Groq API key from environment variables.
# - It creates a ChatGroq model instance using that API key.
# - It sends messages to the model using HumanMessage and AIMessage objects.
# - The config dictionary specifies a session ID to keep track of conversation history.
# - The model can remember previous messages and respond accordingly.

In [ ]:
Here’s a simple explanation of the code:

- It imports classes to help manage chat history.
- It creates an empty dictionary called `store` to save chat sessions.
- The function `get_session_history(session_id)` checks if a session exists in `store`. If not, it creates a new chat history for that session.
- `RunnableWithMessageHistory` wraps your chatbot model so it can remember messages from each session, using the `get_session_history` function to keep track of conversations.

In [9]:
config={"configurable":{"session_id":"chat1"}}

In [10]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Abhii Yadav and I am a Generative AI LLM Engineer")],
    config=config
)

In [11]:
response.content

"Hi Abhii Yadav, it's nice to meet you!\n\nThat's an exciting field to be in.  \n\nWhat kind of generative AI models are you working with? Do you have any particular projects you're passionate about? I'm always eager to learn more about the innovative work being done in this area.\n"

In [12]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Abhii Yadav, as you told me at the beginning of our conversation. 😊  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 111, 'total_tokens': 145, 'completion_time': 0.061818182, 'prompt_time': 0.007343561, 'queue_time': 0.264572129, 'total_time': 0.069161743}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--36ffb607-88e2-41b0-a30b-917272b3ec16-0', usage_metadata={'input_tokens': 111, 'output_tokens': 34, 'total_tokens': 145})

In [13]:
with_message_history.invoke(
    [HumanMessage(content="What do I do?")],
    config=config,
)

AIMessage(content="You said you are a Generative AI LLM Engineer!  \n\nThat's a pretty cool job.  Do you want to tell me more about what that entails?  What kind of projects do you work on? \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 158, 'total_tokens': 207, 'completion_time': 0.089090909, 'prompt_time': 0.026082598, 'queue_time': 0.26320979099999997, 'total_time': 0.115173507}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--7115e2a0-0d16-424f-a74e-b28e27ebd4ef-0', usage_metadata={'input_tokens': 158, 'output_tokens': 49, 'total_tokens': 207})

In [15]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"Since I don't have access to past conversations, I don't know your name.  \n\nIf you'd like to tell me, I'm happy to learn it! 😊  \n\n"

In [16]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you.  \n\nWhat can I do for you today? 😊  \n\n"

In [17]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John!  😄 \n\nIs there anything else I can help you with? \n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [ ]:
### Prompt Template Explained (Simple Version)

**What is a Prompt Template?**
A prompt template is a way to format and organize the instructions and messages you send to a language model (LLM). 
It helps you control how the model responds by setting up a structure for your conversation.

**How Does It Work?**
- You create a template that includes system instructions (like "You are a helpful assistant") and placeholders for user messages.
- When a user sends a message, the template fills in the placeholders with the actual content.
- The formatted prompt is sent to the LLM, which generates a response based on the template and the messages.

**Workflow:**
1. Define a prompt template with instructions and placeholders.
2. Collect user messages.
3. Fill the template with the messages.
4. Send the completed prompt to the LLM.
5. Get the response and show it to the user.

**Use Case:**
Prompt templates are useful for chatbots, customer support, tutoring apps, or any application where you want consistent and controlled responses from an LLM.

**Example:**
Suppose you want your chatbot to always be polite and answer in English. You can set up a prompt template like this:


In [18]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [19]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Abhii")]})

AIMessage(content="Hi Abhii, it's nice to meet you! \n\nI'm happy to help with any questions you have. Just ask away! 😊  What's on your mind? \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 32, 'total_tokens': 76, 'completion_time': 0.08, 'prompt_time': 0.002291697, 'queue_time': 0.27131834200000005, 'total_time': 0.082291697}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--d8c160bb-4a5b-4307-a8e5-9e58447fd019-0', usage_metadata={'input_tokens': 32, 'output_tokens': 44, 'total_tokens': 76})

In [20]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [21]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Abhii")],
    config=config
)

response

AIMessage(content="Hi Abhii! It's nice to meet you.  \n\nI'm ready to answer your questions to the best of my ability. What can I help you with today? 😄  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 32, 'total_tokens': 75, 'completion_time': 0.078181818, 'prompt_time': 0.002296566, 'queue_time': 0.261679563, 'total_time': 0.080478384}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--f6482435-7236-41b9-b4d9-caa439764f42-0', usage_metadata={'input_tokens': 32, 'output_tokens': 43, 'total_tokens': 75})

In [22]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Abhii!  I remember that you told me. 😊 \n\n\n\n'

In [23]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [24]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Abhii")],"language":"Hindi"})
response.content

'नमस्ते Abhii! \n\nमुझे खुशी है कि आपने मुझे ढूंढा। आप मुझसे क्या जानना चाहते हैं? \n\nमैं आपकी सहायता करने के लिए यहाँ हूँ।  😊\n\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [25]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [26]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Abhii")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते Abhii! 😊 \n\nमैं आपकी सहायता करने के लिए तैयार हूँ। आप मुझसे जो भी पूछना चाहें, मैं अपनी पूरी कोशिश करूँगा। \n\nआप क्या जानना चाहते हैं?\n\n'

In [27]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [28]:
response.content

'आपका नाम Abhii है।  😊 \n\n\n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [1]:
%pip install --upgrade --force

Note: you may need to restart the kernel to use updated packages.


ERROR: You must give at least one requirement to install (see "pip help install")


In [2]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [8]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,  # use model from previous cell
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

NameError: name 'model' is not defined

In [40]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal preferences like your favorite ice cream flavor.  \n\nWhat's your favorite ice cream? 😊🍦\n"

In [41]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2" 😊  \n\n\n\n'

In [42]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [43]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n"

In [44]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"